In [1]:
import pandas as pd
dataset = input("Enter the path to the dataset CSV file: ")
concrete = pd.read_csv(dataset)
dependent = input("Enter the name of the dependent column: ")
concrete_temp = concrete.copy()
X = concrete_temp.drop(dependent, axis=1).copy()
# Now, try accessing the column again
y = concrete[dependent]
# Replace 'your_file.csv' with the actual path to your CSV file
frequent_itemset_file_path = input("Enter the path to the frequent itemset CSV file: ")
# Define your column names
column_names = ['Frequent_Itemsets_CHARM']
# Read the CSV file into a DataFrame with specified column names
freq_item = pd.read_csv(frequent_itemset_file_path, names=column_names)
one_hot_encoded = pd.get_dummies(freq_item['Frequent_Itemsets_CHARM'].str.split(expand=True), prefix='', prefix_sep='')
from sklearn.impute import SimpleImputer
def handle_missing_values(dataset):
    # Check if there are missing values in the dataset
    if dataset.isnull().any().any():
        # Identify numerical and categorical columns
        numerical_cols = dataset.select_dtypes(include=['number']).columns
        categorical_cols = dataset.select_dtypes(exclude=['number']).columns

        # Impute numerical columns with mean
        imputer_numeric = SimpleImputer(strategy='mean')
        dataset[numerical_cols] = imputer_numeric.fit_transform(dataset[numerical_cols])

        # Impute categorical columns with the most frequent value (mode)
        imputer_categorical = SimpleImputer(strategy='most_frequent')
        dataset[categorical_cols] = imputer_categorical.fit_transform(dataset[categorical_cols])

        return dataset
    else:
        # If no missing values, return the original dataset
        return dataset
X = handle_missing_values(X)
def binarize_dataframe(input_df, num_bins):
    df = pd.DataFrame(input_df)
    df_binarized = pd.DataFrame()

    for column in df.columns:
        column_name = column+'_binarized'
        bins = pd.qcut(df[column], q=num_bins, labels=False, duplicates='drop')
        df_binarized[column_name] = (bins == bins.max()).astype(int)

    return df_binarized
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

def create_intervals_df(data_frame, column_name, num_intervals):
    # Extract the specified column
    value_ag = data_frame[column_name]

    #print("Extracted column values:")
    #print(value_ag)

    # Check if the column contains numeric data
    if not pd.api.types.is_numeric_dtype(value_ag):
        raise ValueError(f"The '{column_name}' column must contain numeric data.")

    # Check for missing values in the column
    if value_ag.isnull().any():
        raise ValueError(f"The '{column_name}' column contains missing values. Please handle them before processing.")

    # Check if scaling and clipping is necessary
    if value_ag.min() < 0 or value_ag.max() > 1:
        # Rescale to the range [0, 1]
        value_ag_scaled = (value_ag - value_ag.min()) / (value_ag.max() - value_ag.min())

        # Clip values to the range [0, 1]
        value_ag_scaled_clipped = np.clip(value_ag_scaled, 0, 1)
    else:
        # No need to rescale or clip
        value_ag_scaled_clipped = value_ag

    #print("Scaled and clipped column values:")
    #print(value_ag_scaled_clipped)

    # Calculate bin edges dynamically within the [0, 1] range
    bin_edges = np.linspace(0, 1, num=num_intervals + 1)

    # Create intervals and assign labels for clipped values
    data_inter_clipped = pd.cut(value_ag_scaled_clipped.values, bins=bin_edges, labels=[chr(ord('A') + i) for i in range(len(bin_edges) - 1)])

    #print("Computed intervals:")
    #print(data_inter_clipped)

    # Store intervals in a DataFrame
    intervals_df = pd.DataFrame({
        'Category': data_inter_clipped.categories,
        'Min_Value': [value_ag_scaled_clipped[data_inter_clipped == category].min() for category in data_inter_clipped.categories],
        'Max_Value': [value_ag_scaled_clipped[data_inter_clipped == category].max() for category in data_inter_clipped.categories]
    })

    return intervals_df
num_intervals = 10
dependent_column_name = dependent
intervals_df = create_intervals_df(concrete, dependent_column_name, num_intervals)
def categorize_column(dataset, dependent_column_name, num_intervals):
    # Fill NaN values with the mean of the column
    dataset_filled = dataset.fillna(dataset[dependent_column_name].mean())

    # Extract the specified dependent column
    value_ag_filled = dataset_filled[dependent_column_name]

    # Define the target range (0-1)
    target_range = (0, 1)

    # Scale to the target range [0, 1] using np.interp
    value_ag_scaled_filled = np.interp(value_ag_filled, (value_ag_filled.min(), value_ag_filled.max()), target_range)

    # Create intervals and assign labels for scaled values
    data_inter_filled = pd.cut(value_ag_scaled_filled, bins=num_intervals, labels=[chr(ord('A') + i) for i in range(num_intervals)])

    # Store intervals in a DataFrame
    result_df = pd.DataFrame({
        'Category': data_inter_filled,
        dependent_column_name: value_ag_scaled_filled
    })

    return result_df
num_intervals = 10
result_df = categorize_column(concrete, dependent, num_intervals)
#Data Binary
df = pd.DataFrame(concrete)
num_bins = 15
# Use the previously defined binarize_dataframe function
df_binarized = binarize_dataframe(df, num_bins)
dependent_column_name = dependent
# Rename columns and create 'Data_binary'
df_binarized.columns = df_binarized.columns.str.replace('_binarized', '')
data_incidence_input = df_binarized.drop(columns=[dependent_column_name])
Data_binary = data_incidence_input.copy()
Data_binary['Category'] = result_df['Category']
Data_binary[dependent_column_name] = df_binarized[dependent_column_name]
#Jaccard similarity
import pandas as pd
import numpy as np

def calculate_jaccard_similarity(Data_binary, one_hot_encoded):
    # Number of rows in the Data_binary DataFrame
    num_rows_binary = Data_binary.shape[0]

    # Number of rows in the one_hot_encoded DataFrame
    num_rows_encoded = one_hot_encoded.shape[0]

    # Number of columns in the one_hot_encoded DataFrame
    num_cols_encoded = one_hot_encoded.shape[1]

    # Creating a matrix to store Jaccard similarities
    result_matrix = np.zeros((num_rows_binary, num_rows_encoded))

    # Calculating Jaccard similarity for each cell (pair of row in Data_binary and row in one_hot_encoded)
    for i in range(num_rows_binary):
        for j in range(num_rows_encoded):
            set_row_binary = set(Data_binary.iloc[i, 1:])  # Exclude the 'Category' column
            set_row_encoded = set(one_hot_encoded.iloc[j, :])
            intersection_size = len(set_row_binary.intersection(set_row_encoded))
            union_size = len(set_row_binary.union(set_row_encoded))
            similarity = intersection_size / union_size if union_size != 0 else 0
            result_matrix[i, j] = similarity

    # Convert the result matrix to a DataFrame
    result_df = pd.DataFrame(result_matrix, columns=one_hot_encoded.index)

    return result_df
jaccard_result = calculate_jaccard_similarity(Data_binary, one_hot_encoded)
#Transform Matrix
def transform_matrix(matrix):
    # Create a copy of the input matrix
    transformed_matrix = matrix.copy()

    # Apply the transformation
    transformed_matrix[transformed_matrix > 0.50] = 1
    transformed_matrix[transformed_matrix <= 0.50] = 0
    return transformed_matrix
transformed_matrix = transform_matrix(jaccard_result)


In [2]:
# # first we try to add box_type inequalities
# from scipy.sparse import csr_matrix
# from scipy.optimize import minimize
# from scipy.optimize import Bounds
# SX = csr_matrix(transformed_matrix)
# def objective_function(weights, SX, OS):
#     P=np.exp(SX.dot(weights))
#     P/=np.sum(P)
#     loss=-np.sum(OS*np.log(P+1e-10))
#     return loss

# def gradient_function(weights,SX,OS):
#     P=np.exp(SX.dot(weights))
#     P/=np.sum(P)
#     gradients=SX.T@(P - OS)
#     return gradients
# #applying box_type inequality constraints
# #we determine A and B using single width estimation
# features=SX.shape[1]
# sample=SX.shape[0]
# #we chose W as 1
# A=np.full(features,1/sample)
# B=np.full(features,1/sample)
# # calculating A,B using bayesian method
# def optimize_weights(SX,OS,A,B):
#     initial_weights=np.random.normal(0, 0.01, size=SX.shape[1])
#     bounds=Bounds(-B,A)
#     result=minimize(
#         fun=objective_function,
#         x0=initial_weights,
#         args=(SX,OS),
#         method='L-BFGS-B',
#         jac=gradient_function,
#         bounds=bounds
#     )
#     return result.x 


In [3]:
#this is for 1-norm penalozation

from scipy.sparse import csr_matrix
from scipy.optimize import minimize
from scipy.optimize import Bounds
SX = csr_matrix(transformed_matrix)
nfeatures=SX.shape[1]
gamma=np.random.uniform(0, 0.1,nfeatures)
delta=np.random.uniform(0, 0.1,nfeatures)
C1=1
def objective_function(weights, SX, OS,gamma,delta,C1):
    P=np.exp(SX.dot(weights))
    P/=np.sum(P)
    loss=-np.sum(OS*np.log(P+1e-10))
    extra=-np.sum(gamma**2+delta**2)/(2*C1)
    return loss+extra
# def objective_function(weights, SX, OS,gamma,delta,C1):
#     P=np.exp(SX.dot(weights))
#     P/=np.sum(P)
#     loss=-np.sum(OS*np.log(P+1e-10))
#     extra=-np.sum(np.abs(gamma)+np.abs(delta))/(2*C1)
#     return loss+extra
def gradient_function(weights,SX,OS,gamma,delta,C1):
    P=np.exp(SX.dot(weights))
    P/=np.sum(P)
    gradients=SX.T@(P-OS)
    return gradients
#applying box_type inequality constraints
#we determine A and B using single width estimation
features=SX.shape[1]
sample=SX.shape[0]
#we chose W as 1
A=np.full(features,1/sample)
B=np.full(features,1/sample)
# calculating A,B using bayesian method
def optimize_weights(SX,OS,A,B,gamma,delta,C1):
    initial_weights=np.random.normal(0,0.01,size=SX.shape[1])
    bounds=Bounds(-B-gamma,A+delta)
    result=minimize(
        fun=objective_function,
        x0=initial_weights,
        args=(SX, OS, gamma, delta, C1),
        method='L-BFGS-B',
        jac=gradient_function,
        bounds=bounds
    )
    return result.x


In [4]:
# from scipy.sparse import csr_matrix
# from scipy.optimize import minimize
# SX = csr_matrix(transformed_matrix)
# def objective_function(weights, SX, OS):
#     P=np.exp(SX.dot(weights))
#     P/=np.sum(P)
#     loss=-np.sum(OS*np.log(P+1e-10))
#     return loss

# def gradient_function(weights,SX,OS):
#     P=np.exp(SX.dot(weights))
#     P/=np.sum(P)
#     gradients=SX.T@(P - OS)
#     return gradients
# #applying box_type inequality constraints

# def optimize_weights(SX,OS):
#     initial_weights=np.random.normal(0, 0.01, size=SX.shape[1])
#     result=minimize(
#         fun=objective_function,
#         x0=initial_weights,
#         args=(SX,OS),
#         method='L-BFGS-B',
#         jac=gradient_function
#     )
#     return result.x 


In [5]:
# # using gaussian MAP estimation
# from scipy.sparse import csr_matrix
# from scipy.optimize import minimize
# from scipy.optimize import Bounds
# SX = csr_matrix(transformed_matrix)
# nfeatures=SX.shape[1]
# # we chose sigma as 1
# def objective_function(weights, SX, OS,sigma):
#     P=np.exp(SX.dot(weights))
#     P/=np.sum(P)
#     loss=-np.sum(OS*np.log(P+1e-10))
#     regularization=-np.sum(np.abs(weights))/sigma
#     return loss+regularization
# def gradient_function(weights,SX,OS,sigma):
#     P=np.exp(SX.dot(weights))
#     P/=np.sum(P)
#     gradients=SX.T@(P-OS)
#     return gradients
# features=SX.shape[1]
# sample=SX.shape[0]

# def optimize_weights(SX,OS,sigma):
#     initial_weights=np.random.normal(0,0.01,size=SX.shape[1])
#     result=minimize(
#         fun=objective_function,
#         x0=initial_weights,
#         args=(SX, OS,sigma),
#         method='L-BFGS-B',
#         jac=gradient_function
#     )
#     return result.x
# sigma=1

In [6]:
# #using bayesian method
# #adding 1-norm penalization
# from scipy.sparse import csr_matrix
# from scipy.optimize import minimize, Bounds
# import numpy as np

# SX=csr_matrix(transformed_matrix)
# nfeatures=SX.shape[1]
# gamma=np.random.uniform(0, 0.1,nfeatures)
# delta=np.random.uniform(0, 0.1,nfeatures)
# C1=1
# # Define objective function
# def objective_function(weights, SX, OS,gamma,delta,C1):
#     P=np.exp(SX.dot(weights))
#     P/=np.sum(P)
#     loss=-np.sum(OS*np.log(P+1e-10))
#     # extra=-np.sum(gamma**2+delta**2)/(2*C1)
#     extra=-np.sum(np.abs(gamma)+np.abs(delta))/(2*C1)
#     return loss+extra

# # Define gradient of the objective function
# def gradient_function(weights, SX, OS,gamma,delta,C1):
#     P=np.exp(SX.dot(weights))
#     P/=np.sum(P)
#     gradients=SX.T@(P-OS)
#     return gradients

# W=1
# feature_counts=np.array(SX.sum(axis=0)).flatten()  
# L=SX.shape[0]  
# variances=(feature_counts*(1-feature_counts/L))/L

# #Calculate A and B
# A=W*np.sqrt(variances)
# B=W*np.sqrt(variances)

# def optimize_weights(SX, OS, A, B,gamma,delta,C1):
#     initial_weights = np.random.normal(0, 0.01, size=SX.shape[1])
#     bounds=Bounds(-B-gamma, A+delta)
#     result=minimize(
#         fun=objective_function,
#         x0=initial_weights,
#         args=(SX, OS,gamma,delta,C1),
#         method='L-BFGS-B',
#         jac=gradient_function,
#         bounds=bounds
#     )
#     return result.x





In [7]:
def calculate_weights_for_category(category, Data_binary, one_hot_encoded,A,B,gamma,delta,C1):
    jaccard_similarity_matrix=calculate_jaccard_similarity(Data_binary, one_hot_encoded)
    transformed_matrix = transform_matrix(jaccard_similarity_matrix)
    SXmat = csr_matrix(transformed_matrix)

    # Optimize weights using LMVM
    weights = optimize_weights(SXmat, np.ones(SXmat.shape[0]) / SXmat.shape[0],A,B,gamma,delta,C1)
    return weights
categories_weights = {}
categories=Data_binary['Category'].unique()
for category in categories:
    weights = calculate_weights_for_category(category, Data_binary, one_hot_encoded,A,B,gamma,delta,C1)
    categories_weights[category] = {'weights': weights}
# Function to calculate P(x/c) after weights are optimized
def calculate_x_by_c(X, one_hot_encoded, category_weights,A,B,gamma,delta,C1):
    jaccard_result = calculate_jaccard_similarity(X, one_hot_encoded)
    x_by_c = pd.DataFrame(index=X.index)

    # Resetting index for consistency
    X.reset_index(drop=True, inplace=True)
    jaccard_result.reset_index(drop=True, inplace=True)

    for index, row in X.iterrows():
        try:
            category = row['Category']
            jaccard_row = jaccard_result.iloc[index].values.reshape(1, -1)

            category_info = category_weights.get(category)
            if category_info is None:
                raise ValueError(f"No weights found for category: {category}")
            weights = category_info['weights']
            weights = np.expand_dims(weights, axis=0)

            # Perform matrix multiplication directly with weights
            row_value = np.dot(jaccard_row, weights.T).squeeze()

            # Store result
            x_by_c.loc[index, 'P(x/c)'] = row_value

        except ValueError as e:
            print(f"Error in row {index}: {e}")
            raise

    return x_by_c
x_by_c_values = calculate_x_by_c(Data_binary, one_hot_encoded, categories_weights,A,B,gamma,delta,C1)

# Calculate category probabilities
def calculate_category_probabilities(data_frame, category_column):
    category_counts = data_frame[category_column].value_counts()
    category_probabilities = category_counts / len(data_frame)

    return category_counts, pd.DataFrame({
        category_column: category_probabilities.index,
        'Probability': category_probabilities.values
    })
category_counts, category_probabilities_df= calculate_category_probabilities(Data_binary, 'Category')
# Calculate maximum posterior category
categories = np.sort(Data_binary['Category'].unique())

# Set constant values
record_prob = 1 / len(Data_binary)

# Calculate record_prob / P(c) for each category
record_prob_per_category = pd.DataFrame({
    'Category': categories,
    'Record_Prob_Per_Category': record_prob / category_probabilities_df.loc[category_probabilities_df['Category'].isin(categories), 'Probability'].values
})
def calculate_max_posterior_category(binary_dataset, x_by_c_values, record_prob_per_category):
    predicted_categories = []
    for index, row_values in x_by_c_values.iterrows():
        p_x_c = row_values['P(x/c)']
        category_probs = p_x_c * record_prob_per_category['Record_Prob_Per_Category']
        predicted_category = record_prob_per_category['Category'][category_probs.idxmax()]
        predicted_categories.append(predicted_category)

    binary_dataset['Predicted_Category'] = predicted_categories
    return binary_dataset

# Calculate RMSE for the predictions

result = calculate_max_posterior_category(Data_binary, x_by_c_values, record_prob_per_category)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Data_binary, Data_binary[dependent], test_size=0.3, random_state=42)
def calculate_rmse(X_test, intervals_df):
    prediction = calculate_max_posterior_category(Data_binary, x_by_c_values, record_prob_per_category)
    predicted_column = prediction['Predicted_Category']

    sum_squared_error = 0.0
    for i, row in X_test.iterrows():
        actual_category = row['Category']
        predicted_category = predicted_column[i]

        actual_max_value = intervals_df[intervals_df['Category'] == actual_category]['Max_Value'].values[0]
        predicted_max_value = intervals_df[intervals_df['Category'] == predicted_category]['Max_Value'].values[0]

        squared_difference = (predicted_max_value - actual_max_value) ** 2
        sum_squared_error += squared_difference

    mean_squared_error = sum_squared_error / len(X_test)
    rmse = np.sqrt(mean_squared_error)
    return rmse
result_rmse = calculate_rmse(X_test, intervals_df)
print("Root Mean Squared Error (RMSE):", result_rmse)

Root Mean Squared Error (RMSE): 0.8831092375943906


On applying inequality constraints the rmse value decreased 
on applying l2 norm and l1 norm the rmse slightly decreased